# TP Métaheuristiques

## 1ère étape

### Lecture des jeux de données

https://homepages.laas.fr/huguet/drupal/content/métaheuristiques-4ir-info#overlay-context=content/m%25C3%25A9ta-heuristiques-4ir-info  

Remarque : Les arcs enregistrés dans les fichiers ne sont pas forcément dans le bon sens.
Exemple : 4 2 <=> 2 4

Remarque 2 : Transformer les string en int pour manipuler tout plus simplement. 

Remarque 3 : Je pense qu'on peut enlever les arcs inutiles, en vérifiant si ils sont dans un des chemins d'évacuation des noeudsE.
    --> Penser alors à modifier la valeur de graphe.nombreArc ..

In [5]:
# Objets

class Arbre:
    # Liste des noeuds à évacuer
    # Nombre de noeuds à évacuer
    # Le noeud sur
    
    def __init__(self):
        self.listeNoeudsE = []
    def sur(self, c):
        self.nombreNoeudsE = c[0]
        self.noeudSur = c[1]
    def ajoutNoeudE(self, c):
        nouveauNoeudE = NoeudAEvacuer(c)
        self.listeNoeudsE.append(nouveauNoeudE)
        
class NoeudAEvacuer:
    # ID/Population/taux d'évacuation max
    # Chemin d'évacuation
    def __init__(self, c):
        self.nom = c[0]
        self.pop = c[1]
        self.tauxE = c[2]
        self.cheminE = c[4:]
        
class Graphe: 
    # Liste d'arcs UTILES
    # Nombre d'arcs UTILES
    def __init__(self):
        self.listeArcs = []
    def nombre(self,c):
        self.nombreArcs = c[1]
    def ajoutArc(self, c):
        nouveauArc = Arc(c)
        self.listeArcs.append(nouveauArc)
        # On ajoute le même arc avec ses bornes inversées.
        x = c[0]
        c[0] = c[1]
        c[1] = x
        nouveauArc = Arc(c)
        self.listeArcs.append(nouveauArc)

class Arc: 
    # Arc origine/destination(int) ??  /Durée(int)/Capa(int) 
    def __init__(self, c):
        self.origine = c[0]
        self.destination = c[1]
        self.duree = c[3]
        self.capa = c[4]
        
# ------------------------------------

def lecture(nomFichier): 
    with open("./Instances/" + nomFichier, "r") as fichier :
        print ("Name of the file: " + fichier.name)

        # Liste des lignes du fichier
        lignes = fichier.readlines()
        
        # Traitement de l'instance
        val = 1
        for ligne in lignes[1:]: 
            c = ligne.split()
            if c[0] == 'c': 
                val = 2
            else:
                if val == 1:
                    traitementArbre(c)
                elif val == 2:
                    traitementGraphe(c)
                else :
                    print("Erreur")
                    
        # Suppression des arcs inutiles
        majArcsUtiles()
        # Fermeture du fichier
        fichier.close()
        # Fin de la lecture

# ---------------------------------

# Fonctions

def traitementArbre(c):
    if len(c) != 2:
        arbre.ajoutNoeudE(c)
    else:
        arbre.sur(c)
        
def traitementGraphe(c):
    if len(c) != 2: 
        graphe.ajoutArc(c)
    else:
        graphe.nombre(c)

def majArcsUtiles():    
    # Création d'une liste d'arcs vraiment utiles issus de l'arbre.
    arcsUtiles = []
    for noeud in arbre.listeNoeudsE:
        arc = [noeud.nom, noeud.cheminE[0]]
        if arc not in arcsUtiles:
            arcsUtiles.append(arc)
        i = 0
        while True:
            arc = [noeud.cheminE[i], noeud.cheminE[i+1]]
            if arc not in arcsUtiles:
                arcsUtiles.append(arc)
            i = i + 1
            if(noeud.cheminE[i] == arbre.noeudSur):
                break
        
    # Comparaison des arcs du graphe avec la liste arcsUtiles.
    nouveauxArcs = []
    for arcPossible in graphe.listeArcs:
        if [arcPossible.origine,arcPossible.destination] in arcsUtiles:
            nouveauxArcs.append(arcPossible)
            
    graphe.listeArcs = nouveauxArcs
    graphe.nombreArcs = len(graphe.listeArcs)

    
def affichage():
    print("")
    print("=============================================")
    print("--- Programme d'évacuation ---")
    print("Noeud sûr : %s." %(arbre.noeudSur))
    print("Nombre de noeuds à évacuer : %s." %(arbre.nombreNoeudsE)) 
    print("ID - Pop - TauxE (+ Chemin)")
    for noeudE in arbre.listeNoeudsE:
        print("%s     %s     %s" %(noeudE.nom, noeudE.pop, noeudE.tauxE))
        print(*noeudE.cheminE, sep = " ==> ")
        print("------------------------------------")
    print("")
    print("--- A R B R E ---")
    print("Nombre d'arcs : %s." %(graphe.nombreArcs))
    print("Orig/Dest -- Durée -- Capacité")
    for arc in graphe.listeArcs:
        print("  %s/%s       %s      %s" %(arc.origine, arc.destination, arc.duree, arc.capa))
        print("------------------------------------")
    print("=============================================")    

In [6]:
arbre = Arbre()
graphe = Graphe()

lecture("exemple_cours.txt")
affichage()

Name of the file: ./Instances/exemple_cours.txt

--- Programme d'évacuation ---
Noeud sûr : 6.
Nombre de noeuds à évacuer : 3.
ID - Pop - TauxE (+ Chemin)
1     48     8
4 ==> 5 ==> 6
------------------------------------
2     30     5
4 ==> 5 ==> 6
------------------------------------
3     33     3
5 ==> 6
------------------------------------

--- A R B R E ---
Nombre d'arcs : 5.
Orig/Dest -- Durée -- Capacité
  1/4       7      8
------------------------------------
  2/4       4      5
------------------------------------
  3/5       6      3
------------------------------------
  4/5       9      10
------------------------------------
  5/6       12      11
------------------------------------


### Vérification et évaluation d’une solution

#### Données de l'exemple

cf exemple_cours.txt

#### Fichier solution de l'exemple

cf exemple_cours_solution.txt

#### Vérification de la solution

Pour chaque noeud à évacuer, il faut vérifier si le taux d'évacuation est inférieur à la capacité. 
On retrouve le nombre de personnes à évacuer que l'on divise par le taux d'évacuation du fichier solution. Ca donne la durée d'évacuation du noeud de départ. On y ajoute la date de début d'évacuation du noeud associé. 
Il faut ensuite calculer pour chaque noeud à évacuer la durée du chemin d'évacuation, et y ajouter la valeur trouvée précedemment. 
La valeur de la fonction objectif est le max de ces valeurs. 

In [3]:
def lecture_solution(nom_fichier_solution): 

def verification(nom_fichier, nom_fichier_solution):
    lecture(nom_fichier)
    result = "vrai"
    max = 0
    for noeudE in arbre.listeNoeudsE:
        
        
    return result
        
        
        
        
        
        
        self.nom = c[0]
        self.pop = c[1]
        self.tauxE = c[2]
        self.cheminE = c[4:]

In [7]:
arbre = Arbre()
graphe = Graphe()
solution = Solution()

verification(exemple_cours.txt, exemple_cours_solution.txt)